In [1]:
import os
import sys
sys.path.append("../")
import random

import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.models import Model

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from layer.bayesian_dropout_layer import BayesianDropoutLayer
from collections import Counter


import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2022-04-02 20:56:18.233207: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-02 20:56:18.265147: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-02 20:56:18.265386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
train_folder = "/home/fsantiago/projects/medium/datasets/cifar-10/train_aug"
train_labels = "/home/fsantiago/projects/medium/datasets/cifar-10/labels_aug.csv"

In [3]:
labels_df = pd.read_csv(train_labels, index_col="id")
one_hot = OneHotEncoder().fit(labels_df[["label"]])
one_hot_labels = OneHotEncoder().fit_transform(labels_df[["label"]]).toarray()

In [4]:
auto_input = layers.Input(shape=(32, 32, 3))
layer_1 = BayesianDropoutLayer(units=512, dropout=0.1, activation=tf.nn.relu)
layer_2 = BayesianDropoutLayer(units=512, dropout=0.1, activation=tf.nn.relu)
layer_3 = BayesianDropoutLayer(units=128, dropout=0.1, activation=tf.nn.relu)

def get_conv_layer(filters, x, dropout=0.25, kernel_size=(3, 3)):

    x = layers.Conv2D(filters, kernel_size=kernel_size, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(dropout)(x)
    return x

def get_conv_layer_2(filters, x, number_of_layers):

    for i in range(number_of_layers - 1):
        x = layers.Conv2D(filters, kernel_size=(3,3), padding="same")(x)

    x = layers.Conv2D(filters, kernel_size=(3,3), strides=(2, 2), padding="same")(x)
    return x


def get_mini_vgg_19():

    x = get_conv_layer(32, auto_input, kernel_size=(3, 3))
    x = get_conv_layer(32, x, kernel_size=(3, 3))
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = get_conv_layer(64, x)
    x = get_conv_layer(64, x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = get_conv_layer(128, x)
    x = get_conv_layer(128, x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = layers.Flatten()(x)

    # x = layer_1(x)
    # x = layer_2(x)
    # x = layer_3(x)

    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(10, activation='softmax')(x)

    return x


def get_mini_plain():

    x = get_conv_layer(64, auto_input, 3)
    x = get_conv_layer(128, x, 4)
    x = get_conv_layer(256, x, 6)
    x = get_conv_layer(512, x, 2)

    x = layers.AveragePooling2D((2, 2), padding="same")(x)

    x = layers.Flatten()(x)

    x = layers.Dense(1000, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(10, activation='softmax')(x)

    return x

x = get_mini_vgg_19()
model = Model(auto_input, x)


2022-04-02 20:56:18.545081: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-02 20:56:18.546141: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-02 20:56:18.546499: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-02 20:56:18.546790: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [5]:
test_folder = "/home/fsantiago/projects/medium/datasets/cifar-10/test/test/"
model.load_weights("./bayesian_weigths/weights.ckpt")

img_number = 300000
files_index = list(range(1, img_number + 1))
test_labels_df = pd.DataFrame()
imgs = []

for file_idx in tqdm(files_index):
    file = f"{test_folder}{file_idx}.png"
    img = plt.imread(file)
    imgs.append(img)

100%|██████████| 300000/300000 [00:46<00:00, 6413.39it/s]


In [6]:
def load_model():
    x = get_mini_vgg_19()
    model = Model(auto_input, x)
    model.load_weights("./normal_nn_weigths/weights.ckpt")

    return model

def get_predictions(one_hot, i, imgs, chunk):
    model = load_model()
    preds_onehot = model(np.array(imgs[i*chunk:(i+1)*chunk]))
    preds = [pred[0] for pred in one_hot.inverse_transform(preds_onehot)]
    return preds

In [8]:
import gc
chunk = 3000
reps = int(300000 / chunk) 
bayesian_reps = 1
test_labels_df = pd.DataFrame()
tf.keras.backend.clear_session()

for i in tqdm(range(reps)):    
    bayesian_predictions = []
    for j in range(bayesian_reps):
        preds = get_predictions(one_hot, i, imgs, chunk)     
        bayesian_predictions.append(preds)
        del(preds)
        gc.collect()

    bayesian_predictions = np.array(bayesian_predictions).T

    for bay_i in range(len(bayesian_predictions)):
        preds = dict(Counter(bayesian_predictions[bay_i]))
        key = max(preds, key=preds.get)
        probability = preds[key]/bayesian_reps

        new_label = {"id": [i*chunk + bay_i + 1] ,
                     "label": [key], "prob": [probability]}
        new_row = pd.DataFrame.from_records(new_label, index="id")
        test_labels_df = pd.concat([test_labels_df, new_row])

    tf.keras.backend.clear_session()


100%|██████████| 100/100 [24:37<00:00, 14.78s/it]


In [9]:
test_labels_df.to_csv("bayesian_results.csv")

In [10]:
test_labels_df["prob"].mean()

1.0

In [11]:
test_labels_df["label"].to_csv("results_submission.csv")